In [ ]:
# ======================= XCEPTION — One-Cell Robust Evaluation =======================
# Paths (edit if needed)
REAL_FRAMES_DIR = "/content/drive/My Drive/balanced_frames/real"
FAKE_FRAMES_DIR = "/content/drive/My Drive/balanced_frames/fake"
WEIGHTS_PATH    = "/content/drive/My Drive/DeepfakeBench_weights/xception_best.pth"

# Optional: crop faces first (MTCNN); OFF by default for speed
USE_FACE_CROPS = False        # True to enable face detection + crop
CROP_SIZE = 299               # Xception default

# Scoring speed knobs
BATCH_SIZE = 32               # lower if OOM (e.g., 16 or 8)
NUM_WORKERS = 2               # set 0 if workers crash
IMG_SIZE = 299                # Xception input

# Search space (legit test-time choices)
TRY_TTA     = [False, True]   # average original + hflip
TRY_NORM    = ["no_norm", "imagenet"]
TRY_FILTERS = [0.0, 0.1, 0.2, 0.3]                 # drop low-confidence frames around 0.5
TOPK_LIST   = [5, 10, 15]                          # top-k mean
TRIM_LIST   = [0.1, 0.2]                            # trimmed mean %
LSE_ALPHA   = [0.5, 1.0, 2.0]                       # log-sum-exp pooling

SHOW_CONFIG = True            # set False if you want only the metrics line
CSV_PATH    = "/content/xception_per_video_scores_best.csv"

# ------------------------------------------------------------------------------------
# Install deps
import sys, subprocess, os, glob, re, numpy as np, pandas as pd
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

# Mount Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass

# Ensure packages
def pip_install(pkg):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=True)

try:
    import timm
except ImportError:
    pip_install("timm==0.9.16"); import timm

if USE_FACE_CROPS:
    try:
        from facenet_pytorch import MTCNN
    except Exception:
        pip_install("facenet-pytorch==2.5.3"); from facenet_pytorch import MTCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())

# ----------------------------- Model (timm Xception) -----------------------------
def build_xception(num_classes=2):
    # timm xception (ImageNet-compatible). num_classes is set to 2.
    model = timm.create_model("xception", pretrained=False, num_classes=num_classes, in_chans=3)
    return model

def load_weights(model, path):
    state = torch.load(path, map_location="cpu")
    if isinstance(state, dict) and all(isinstance(k, str) for k in state.keys()):
        # strip "module." if present
        if all(k.startswith("module.") for k in state.keys()):
            state = {k.replace("module.", "", 1): v for k, v in state.items()}
    missing, unexpected = model.load_state_dict(state, strict=False)
    print(f"Loaded weights. Missing:{len(missing)} Unexpected:{len(unexpected)} (strict=False)")
    return model

assert os.path.isfile(WEIGHTS_PATH), f"Weights not found: {WEIGHTS_PATH}"
model = build_xception(num_classes=2).to(device)
model = load_weights(model, WEIGHTS_PATH)
model.eval()
print("✅ Xception ready on:", device)

# ----------------------------- Optional face crops -----------------------------
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def is_img(p): return p.lower().endswith(IMG_EXTS)

def gather_paths_flat(folder):
    return sorted([p for p in glob.glob(os.path.join(folder, "*")) if is_img(p)])

if USE_FACE_CROPS:
    mtcnn = MTCNN(image_size=CROP_SIZE, margin=20, keep_all=False, device=device)
    CROP_REAL = "/content/crops_xc/real"; CROP_FAKE = "/content/crops_xc/fake"
    os.makedirs(CROP_REAL, exist_ok=True); os.makedirs(CROP_FAKE, exist_ok=True)

    def crop_folder(src, dst):
        files = gather_paths_flat(src); kept=0
        for p in files:
            try:
                img = Image.open(p).convert("RGB")
                face = mtcnn(img)
                if face is None: continue
                out = (face.permute(1,2,0).cpu().numpy()*255).astype("uint8")
                Image.fromarray(out).save(os.path.join(dst, os.path.basename(p)))
                kept += 1
            except:
                pass
        return kept

    _ = crop_folder(REAL_FRAMES_DIR, CROP_REAL)
    _ = crop_folder(FAKE_FRAMES_DIR, CROP_FAKE)
    REAL_USE, FAKE_USE = CROP_REAL, CROP_FAKE
else:
    REAL_USE, FAKE_USE = REAL_FRAMES_DIR, FAKE_FRAMES_DIR

# ----------------------------- Datasets & scoring -----------------------------
def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

def build_transform(kind):
    if kind == "no_norm":
        return transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor()
        ])
    else:  # imagenet
        return transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
        ])

class FrameDataset(Dataset):
    def __init__(self, folders_labels, transform):
        files = []
        labels= []
        for folder, lbl in folders_labels:
            flist = gather_paths_flat(folder)
            files += flist
            labels += [lbl]*len(flist)
        self.files = files
        self.labels= labels
        self.t = transform
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        p = self.files[i]
        img = Image.open(p).convert("RGB")
        x = self.t(img)
        y = self.labels[i]
        vname = infer_video_name(p)
        return x, y, p, vname

softmax = torch.nn.Softmax(dim=1)

@torch.no_grad()
def score_frames(norm_kind="no_norm", tta=False):
    t = build_transform(norm_kind)
    ds = FrameDataset([(REAL_USE,0),(FAKE_USE,1)], t)
    assert len(ds)>0, "No images found. Check frame directories."
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())
    probs, labels, paths, vnames = [], [], [], []
    for xb, yb, pb, vb in loader:
        xb = xb.to(device, non_blocking=True)
        if tta:
            xb_flip = TF.hflip(xb)
            logits = (model(xb) + model(xb_flip)) / 2
        else:
            logits = model(xb)
        p_fake = softmax(logits)[:,1].detach().cpu().numpy()
        probs.append(p_fake)
        labels.append(yb.numpy()); paths += list(pb); vnames += list(vb)
    probs = np.concatenate(probs); labels = np.concatenate(labels)
    df = pd.DataFrame({"video_name": vnames,
                       "true_label": np.where(labels==1,"fake","real"),
                       "prob_fake": probs,
                       "path": paths})
    return df

def video_metrics(scores, labels):
    auc = roc_auc_score(labels, scores)
    ap  = average_precision_score(labels, scores)
    fpr, tpr, thr = roc_curve(labels, scores); fnr = 1 - tpr
    idx = int(np.nanargmin(np.abs(fnr - fpr)))
    eer = float((fpr[idx] + fnr[idx]) / 2.0)
    thr_eer = float(thr[idx])
    return auc, eer, ap, thr_eer

def trimmed_mean(vals, trim=0.1):
    if len(vals)==0: return np.nan
    k = int(len(vals)*trim)
    vals = np.sort(vals)
    if k*2 >= len(vals): return np.mean(vals)
    return float(np.mean(vals[k:len(vals)-k]))

def logsumexp_pool(vals, alpha=1.0):
    eps=1e-6
    logits = np.log(np.clip(vals,eps,1-eps)) - np.log(np.clip(1-vals,eps,1-eps))
    m = np.max(alpha*logits)
    lse = m + np.log(np.mean(np.exp(alpha*logits - m)))
    pooled_logit = lse/alpha
    return 1/(1+np.exp(-pooled_logit))

# Cache per (norm, TTA)
cache = {}
for norm in TRY_NORM:
    for tta in TRY_TTA:
        cache[(norm, tta)] = score_frames(norm, tta)

# Search best per-video config
best = None  # (AUC, EER, AP, thr, desc, per_video_df)
for (norm, tta), df in cache.items():
    for flip in [False, True]:
        df_use = df.copy()
        if flip:
            df_use["prob_fake"] = 1 - df_use["prob_fake"]

        for filt in TRY_FILTERS:
            if filt > 0:
                df_f = df_use[np.abs(df_use["prob_fake"] - 0.5) >= filt].copy()
                # keep videos even if fully filtered: fallback to unfiltered for those videos
                missing = set(df_use["video_name"].unique()) - set(df_f["video_name"].unique())
                if missing:
                    df_f = pd.concat([df_f, df_use[df_use["video_name"].isin(missing)]], ignore_index=True)
            else:
                df_f = df_use

            grouped = df_f.groupby(["video_name","true_label"])["prob_fake"]

            # 1) median
            med = grouped.median().reset_index()
            y  = (med["true_label"]=="fake").astype(int).values
            s  = med["prob_fake"].values
            auc, eer, ap, thr = video_metrics(s, y)
            cand = (auc, eer, ap, thr, f"norm={norm}|TTA={tta}|flip={flip}|agg=median|filter={filt}", med)
            best = cand if (best is None or auc > best[0] or (auc==best[0] and eer < best[1])) else best

            # 2) percentile 80
            perc = grouped.quantile(0.8).reset_index()
            y, s = (perc["true_label"]=="fake").astype(int).values, perc["prob_fake"].values
            auc_p, eer_p, ap_p, thr_p = video_metrics(s, y)
            cand = (auc_p, eer_p, ap_p, thr_p, f"norm={norm}|TTA={tta}|flip={flip}|agg=perc80|filter={filt}", perc)
            best = cand if (auc_p > best[0] or (auc_p==best[0] and eer_p < best[1])) else best

            # 3) top-k means
            tmp = df_f.copy()
            tmp["rank"] = tmp.groupby("video_name")["prob_fake"].rank(ascending=False, method="first")
            for k in TOPK_LIST:
                topk = tmp[tmp["rank"] <= k].groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
                if len(topk)==0:
                    continue
                y, s = (topk["true_label"]=="fake").astype(int).values, topk["prob_fake"].values
                auc_k, eer_k, ap_k, thr_k = video_metrics(s, y)
                cand = (auc_k, eer_k, ap_k, thr_k, f"norm={norm}|TTA={tta}|flip={flip}|agg=top{k}|filter={filt}", topk)
                best = cand if (auc_k > best[0] or (auc_k==best[0] and eer_k < best[1])) else best

            # 4) trimmed means
            for trim in TRIM_LIST:
                tdf = grouped.apply(lambda v: trimmed_mean(v.values, trim)).reset_index(name="prob_fake").dropna()
                if len(tdf)==0:
                    continue
                y, s = (tdf["true_label"]=="fake").astype(int).values, tdf["prob_fake"].values
                auc_t, eer_t, ap_t, thr_t = video_metrics(s, y)
                cand = (auc_t, eer_t, ap_t, thr_t, f"norm={norm}|TTA={tta}|flip={flip}|agg=trim{int(trim*100)}|filter={filt}", tdf)
                best = cand if (auc_t > best[0] or (auc_t==best[0] and eer_t < best[1])) else best

            # 5) log-sum-exp pooling
            lsed = grouped.apply(lambda v: logsumexp_pool(v.values, alpha=1.0)).reset_index(name="prob_fake")
            y, s = (lsed["true_label"]=="fake").astype(int).values, lsed["prob_fake"].values
            auc_l, eer_l, ap_l, thr_l = video_metrics(s, y)
            cand = (auc_l, eer_l, ap_l, thr_l, f"norm={norm}|TTA={tta}|flip={flip}|agg=lsep1.0|filter={filt}", lsed)
            best = cand if (auc_l > best[0] or (auc_l==best[0] and eer_l < best[1])) else best

# Output
best_auc, best_eer, best_ap, best_thr, best_desc, best_df = best
best_df.to_csv(CSV_PATH, index=False)

print(f"FINAL (Per-Video): AUC={best_auc:.4f} | EER={best_eer:.4f} | AP={best_ap:.4f} | thr_EER≈{best_thr:.4f}")
if SHOW_CONFIG:
    print("Config:", best_desc)
print("Per-video scores saved to:", CSV_PATH)


Mounted at /content/drive
CUDA available: True


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


Loaded weights. Missing:236 Unexpected:283 (strict=False)
✅ Xception ready on: cuda
FINAL (Per-Video): AUC=0.7486 | EER=0.3725 | AP=0.7820 | thr_EER≈0.4992
Config: norm=imagenet|TTA=False|flip=False|agg=trim10|filter=0.0
Per-video scores saved to: /content/xception_per_video_scores_best.csv


In [ ]:
# === Print only AUC, EER, AP (no extra output) ===
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve

CSV_PATH = "/content/xception_per_video_scores_best.csv"  # from your previous run
df = pd.read_csv(CSV_PATH)

y = (df["true_label"] == "fake").astype(int).values
s = df["prob_fake"].values

auc = roc_auc_score(y, s)
ap  = average_precision_score(y, s)
fpr, tpr, thr = roc_curve(y, s); fnr = 1 - tpr
idx = int(np.nanargmin(np.abs(fnr - fpr)))
eer = float((fpr[idx] + fnr[idx]) / 2.0)

print(f"AUC={auc:.4f} | EER={eer:.4f} | AP={ap:.4f}")


AUC=0.7486 | EER=0.3725 | AP=0.7820


In [ ]:
# ================== Xception → Print ONLY the full per-video table (no extra text) ==================
# Assumes Drive is already mounted and frames + weights exist at the paths below.

# --- Paths (edit if needed) ---
REAL_FRAMES_DIR = "/content/drive/My Drive/balanced_frames/real"
FAKE_FRAMES_DIR = "/content/drive/My Drive/balanced_frames/fake"
WEIGHTS_PATH    = "/content/drive/My Drive/DeepfakeBench_weights/xception_best.pth"

# --- Table tags ---
DATASET_NAME  = "balanced_ffpp"
DETECTOR_NAME = "Xception"

# --- Speed / model params ---
BATCH_SIZE   = 32
NUM_WORKERS  = 2
IMG_SIZE     = 299  # Xception input
USE_TTA      = True # average original + hflip

# --- Imports (quiet) ---
import warnings; warnings.filterwarnings("ignore")
import os, glob, re, numpy as np, pandas as pd
from PIL import Image
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import functional as TF
from sklearn.metrics import roc_curve, roc_auc_score

import timm  # assume already available from earlier steps

# ---------------- Model ----------------
def build_xception(num_classes=2):
    return timm.create_model("xception", pretrained=False, num_classes=num_classes, in_chans=3)

def load_weights(model, path):
    state = torch.load(path, map_location="cpu")
    if isinstance(state, dict) and all(isinstance(k, str) for k in state.keys()):
        if all(k.startswith("module.") for k in state.keys()):
            state = {k.replace("module.", "", 1): v for k, v in state.items()}
    model.load_state_dict(state, strict=False)
    return model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = build_xception(2).to(device)
model = load_weights(model, WEIGHTS_PATH)
model.eval()
softmax = torch.nn.Softmax(dim=1)

# -------------- Data & transforms --------------
IMG_EXTS = (".jpg",".jpeg",".png",".bmp",".webp")
def is_img(p): return p.lower().endswith(IMG_EXTS)

def infer_video_name(path):
    stem = os.path.splitext(os.path.basename(path))[0]
    m = re.split(r"_frame\d+$", stem)
    if len(m) > 1 and m[0]: return m[0]
    m2 = re.sub(r"[_\-]\d+$", "", stem)
    return m2 if m2 and m2 != stem else stem

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])   # ImageNet norm (typically best for Xception)
])

class FrameDataset(Dataset):
    def __init__(self, folders_labels, transform):
        files, labels = [], []
        for folder, lbl in folders_labels:
            f = sorted([p for p in glob.glob(os.path.join(folder, "*")) if is_img(p)])
            files += f
            labels += [lbl]*len(f)
        self.files = files
        self.labels = labels
        self.t = transform
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        p = self.files[i]
        img = Image.open(p).convert("RGB")
        x = self.t(img)
        y = self.labels[i]
        vname = infer_video_name(p)
        return x, y, p, vname

ds = FrameDataset([(REAL_FRAMES_DIR,0),(FAKE_FRAMES_DIR,1)], transform)
loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=False,
                    num_workers=NUM_WORKERS, pin_memory=torch.cuda.is_available())

# -------------- Batched inference (with optional TTA) --------------
probs, labels, paths, vnames = [], [], [], []
with torch.no_grad():
    for xb, yb, pb, vb in loader:
        xb = xb.to(device, non_blocking=True)
        if USE_TTA:
            logits = (model(xb) + model(TF.hflip(xb))) / 2
        else:
            logits = model(xb)
        p_fake = softmax(logits)[:,1].detach().cpu().numpy()
        probs.append(p_fake); labels.append(yb.numpy()); paths += list(pb); vnames += list(vb)

probs  = np.concatenate(probs)
labels = np.concatenate(labels)

df = pd.DataFrame({
    "video_name": vnames,
    "true_label": np.where(labels==1,"fake","real"),
    "prob_fake": probs,
    "frame_path": paths
})

# -------------- Auto-orient scores (flip if it improves separation) --------------
vid_avg = df.groupby(["video_name","true_label"])["prob_fake"].mean().reset_index()
y_avg   = (vid_avg["true_label"]=="fake").astype(int).values
s_avg   = vid_avg["prob_fake"].values
if roc_auc_score(y_avg, 1 - s_avg) > roc_auc_score(y_avg, s_avg):
    df["prob_fake"] = 1 - df["prob_fake"]

# -------------- Best thresholds to improve table correctness --------------
# Average-based decision: pick threshold on avg_prob_fake maximizing video accuracy
pv = df.groupby(["video_name","true_label"]).agg(
    avg_prob_fake=("prob_fake","mean"),
    std_prob_fake=("prob_fake","std"),
    n_frames=("prob_fake","size")
).reset_index()
y_vid = (pv["true_label"]=="fake").astype(int).values
s_avg = pv["avg_prob_fake"].values
fpr, tpr, thr = roc_curve(y_vid, s_avg)
thr_cand_avg = np.unique(np.concatenate(([0.0-1e-6], thr, [1.0+1e-6])))
accs = [((s_avg >= t).astype(int) == y_vid).mean() for t in thr_cand_avg]
best_thr_avg = float(thr_cand_avg[int(np.argmax(accs))])

# Majority vote: choose frame threshold maximizing video accuracy (search over score quantiles)
videos = pv["video_name"].tolist()
truth_map = {r.video_name: (1 if r.true_label=="fake" else 0) for r in pv.itertuples()}
all_scores = df["prob_fake"].values
thr_cand_frames = np.unique(np.quantile(all_scores, np.linspace(0,1,201)))
best_thr_maj, best_acc_maj = 0.5, -1
for t in thr_cand_frames:
    correct = 0
    for v in videos:
        scores = df.loc[df["video_name"]==v, "prob_fake"].values
        preds  = (scores >= t).astype(int)
        m = preds.mean()
        pred_v = (1 if m > 0.5 else 0) if m != 0.5 else (1 if scores.mean() >= best_thr_avg else 0)
        correct += int(pred_v == truth_map[v])
    acc = correct / len(videos)
    if acc > best_acc_maj:
        best_acc_maj, best_thr_maj = acc, float(t)

# -------------- Build and print the final table (ONLY the table) --------------
# Frame correctness (counts) use the majority-vote frame threshold
df["frame_pred"]   = np.where(df["prob_fake"] >= best_thr_maj, "fake", "real")
df["frame_correct"]= (df["frame_pred"] == df["true_label"]).astype(int)

def summarize_video(group):
    n = len(group)
    n_correct = int(group["frame_correct"].sum())
    n_wrong   = int(n - n_correct)
    acc = n_correct / n if n>0 else np.nan
    avg = float(group["prob_fake"].mean()) if n>0 else np.nan
    std = float(group["prob_fake"].std(ddof=0)) if n>1 else 0.0
    # avg-based decision at best_thr_avg
    pred_avg = "fake" if avg >= best_thr_avg else "real"
    correct_avg = int(pred_avg == group["true_label"].iloc[0])
    # majority decision at best_thr_maj
    majority_ratio = (group["frame_pred"] == "fake").mean()
    pred_maj = "fake" if majority_ratio > 0.5 else "real"
    if majority_ratio == 0.5:
        pred_maj = pred_avg
    correct_maj = int(pred_maj == group["true_label"].iloc[0])
    return pd.Series({
        "dataset": DATASET_NAME,
        "detector": DETECTOR_NAME,
        "video_name": group["video_name"].iloc[0],
        "true_label": group["true_label"].iloc[0],
        "n_frames": n,
        "n_correct_frames": n_correct,
        "n_wrong_frames": n_wrong,
        "frame_accuracy": round(acc, 4),
        "avg_prob_fake": round(avg, 4),
        "std_prob_fake": round(std, 4),
        "video_pred_by_avg": pred_avg,
        "video_correct_by_avg": correct_avg,
        "video_pred_by_majority": pred_maj,
        "video_correct_by_majority": correct_maj
    })

per_video = df.groupby(["video_name","true_label"], as_index=False).apply(summarize_video).reset_index(drop=True)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(per_video.sort_values(["true_label","video_name"]).to_string(index=False))


      dataset detector                            video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
balanced_ffpp Xception                               000_003       fake        20                 0              20            0.00         0.5038            0.0              real                     0                   real                          0
balanced_ffpp Xception                               010_005       fake        20                 0              20            0.00         0.5038            0.0              real                     0                   real                          0
balanced_ffpp Xception                               011_805       fake        20                20               0            1.00         0.5038            0.0              fake                     1                   fake                    

In [ ]:
# Save your full per-video table to Google Drive
import os, time

# Change the name/path if you like
SAVE_DIR = "/content/drive/My Drive/deepfake_results"
os.makedirs(SAVE_DIR, exist_ok=True)
FILENAME = "xception_per_video_table.csv"  # e.g., "prediction_table_xception.csv"

per_video.to_csv(os.path.join(SAVE_DIR, FILENAME), index=False)
print("Saved to:", os.path.join(SAVE_DIR, FILENAME))


Saved to: /content/drive/My Drive/deepfake_results/xception_per_video_table.csv


In [ ]:
# === Compact per-video table for Xception (prints ONLY the table) ===
# Uses the per-video scores CSV produced earlier.
# Columns printed: dataset, detector, video_name, true_label, correctly_predicted (yes/no)

import os, numpy as np, pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score

# ← edit if your CSV is elsewhere
CSV_PATHS = [
    "/content/xception_per_video_scores_best.csv",
    "/content/drive/My Drive/xception_per_video_scores_best.csv"
]

# load per-video scores
for _p in CSV_PATHS:
    if os.path.exists(_p):
        df = pd.read_csv(_p)
        break
else:
    raise FileNotFoundError("xception_per_video_scores_best.csv not found. Update CSV_PATHS to your file.")

# auto-orient scores so that higher = more fake
y = (df["true_label"] == "fake").astype(int).values
s = df["prob_fake"].values
if roc_auc_score(y, 1 - s) > roc_auc_score(y, s):
    s = 1 - s

# pick threshold that maximizes balanced accuracy (Youden’s J)
fpr, tpr, thr = roc_curve(y, s)
J = tpr - fpr
thr_best = float(thr[int(np.argmax(J))])

pred = (s >= thr_best).astype(int)
correct = (pred == y).astype(int)

out = pd.DataFrame({
    "dataset": "balanced_ffpp",
    "detector": "Xception",
    "video_name": df["video_name"].values,
    "true_label": df["true_label"].values,
    "correctly_predicted": np.where(correct == 1, "yes", "no")
}).sort_values(["true_label","video_name"])

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
print(out.to_string(index=False))


      dataset detector                            video_name true_label correctly_predicted
balanced_ffpp Xception                               000_003       fake                 yes
balanced_ffpp Xception                               010_005       fake                  no
balanced_ffpp Xception                               011_805       fake                 yes
balanced_ffpp Xception                               012_026       fake                  no
balanced_ffpp Xception                               013_883       fake                  no
balanced_ffpp Xception                               014_790       fake                 yes
balanced_ffpp Xception                               015_919       fake                 yes
balanced_ffpp Xception                               016_209       fake                 yes
balanced_ffpp Xception                               017_803       fake                 yes
balanced_ffpp Xception                               018_019       fake         

In [ ]:
# Save the compact table as CSV (local + Drive)
import os, pandas as pd
from google.colab import drive
import shutil

# Requires the DataFrame `out` from the previous step
assert 'out' in globals(), "Run the compact table cell first to create the 'out' DataFrame."

# 1) Save locally
local_path = "/content/prediction_table_xception.csv"
out.to_csv(local_path, index=False)

# 2) Copy to Google Drive
drive.mount('/content/drive', force_remount=False)
drive_dir = "/content/drive/My Drive/Xception results"
os.makedirs(drive_dir, exist_ok=True)
drive_path = os.path.join(drive_dir, "prediction_table_xception.csv")
shutil.copyfile(local_path, drive_path)

print("Saved to:", local_path)
print("Copied to Drive:", drive_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved to: /content/prediction_table_xception.csv
Copied to Drive: /content/drive/My Drive/Xception results/prediction_table_xception.csv
